In [3]:
from datetime import datetime
import time


from powerio.address import PublicAddress
from powerio.keys import PrivateKey
from powerio.api_v2 import ClientII as api2

alice={}
alice_key_str = 
alice['key']=PrivateKey.from_der(alice_key_str)
alice['account'] = ''

bob={}
bob_key_str = 
bob['key'] = PrivateKey.from_der(bob_key_str)
bob['account'] = ''


screen_width=100

def PrintMessage(message_time, message,alignment_type=0):
    time_str=datetime.utcfromtimestamp(message_time/1000).strftime('%Y-%m-%d %H:%M:%S')
    if alignment_type==1:
        time_str=time_str.rjust(screen_width)
        message=message.rjust(screen_width)
    print(time_str)
    print(message)
    print('')
    return 0
          
def PrintAllMessages(message_list,owner_address,recipient_address,first):
    if first==1:
        print(recipient_address.ljust(int(screen_width/2))+'Me'.rjust(int(screen_width/2)))
        print('')
    owner_address_bin=PublicAddress.txt2bin(owner_address)
    for msg_key in sorted(message_list):
        alignment_type=0
        if message_list[msg_key]['from']==owner_address_bin:
            alignment_type=1
        PrintMessage(message_list[msg_key]['time'],message_list[msg_key]['msg'],alignment_type)
    return 0    
          
def NewMessages(recipient, sender):
    recipient_address=recipient['account']
    sender_address=sender['account']
    start_block_hash=recipient['last_block_hash']
    API = api2()
    last_recipient_block_hash=API.get_prev_block_for_addr(recipient_address)
    result={}
    result['result']='false'
    if start_block_hash!=last_recipient_block_hash:
        new_msgs=GetMessages(recipient_address, sender_address,start_block_hash,last_recipient_block_hash)
        if len(new_msgs)!=0:
            result['msg_list']=new_msgs
            result['last_block_hash']=last_recipient_block_hash
            result['result']='true'
    return result

def GetMessageFromBlockByHash(recipient_address, sender_address, block_hash):
    API = api2()
    block_txs=API.get_tx_from_block(block_hash,recipient_address)  
    msg_list={}
    sender_bin=PublicAddress.txt2bin(sender_address)
    for tx in block_txs:
        if 'txext' in block_txs[tx]:
            if block_txs[tx]['to']==sender_bin:
                temp={}
                temp['msg']=''
                if 'msg' in block_txs[tx]['txext']:
                    temp['msg']=block_txs[tx]['txext']['msg']
                temp['time']=block_txs[tx]['t']
                temp['from']=block_txs[tx]['from']
                temp['to']=block_txs[tx]['to']
                msg_list[block_txs[tx]['seq']]=temp
                
    return msg_list
          
def GetMessages(recipient_address, sender_address, start_block_hash, end_block_hash):
    API = api2()
    last_recipient_block=API.get_prev_block_for_addr(recipient_address)
    msg_list={}
    prev_block_hash=end_block_hash
    while prev_block_hash != start_block_hash and prev_block_hash !='':
        new_msg=GetMessageFromBlockByHash(recipient_address, sender_address,prev_block_hash)
        msg_list.update(new_msg)
        prev_block_hash=API.get_prev_block_for_addr(recipient_address,prev_block_hash)
    return msg_list          
          
def SartWatchingChat(owner,recipient):
    owner['last_block_hash']=''       
    recipient['last_block_hash']=''        
    i=0
    while i<100:
        i += 1
        temp_message_list={}
        result=NewMessages(owner, recipient)
        if result['result']=='true':
            temp_message_list.update(result['msg_list'])
            owner['last_block_hash']=result['last_block_hash']        
        result=NewMessages( recipient, owner)
        if result['result']=='true':
            temp_message_list.update(result['msg_list'])
            recipient['last_block_hash']=result['last_block_hash']   
        PrintAllMessages(temp_message_list,owner['account'],recipient['account'],i)    
        time.sleep(2)
    return 0


In [5]:
SartWatchingChat(alice,bob)

AA100000005033269272                                                                              Me

                                                                                 2019-02-13 13:23:02
                                                                                               Hi!!!

2019-02-13 13:23:22
Hello!

                                                                                 2019-02-13 13:33:14
                                                                          I`m Alice. And ho are you?

2019-02-13 13:33:23
I`m Bob. Glad to e-meet you!

                                                                                 2019-02-13 13:33:44
                                                                             What are you doing now?



0